In [ ]:
from procDataSet import TrainingQuery
from bert_serving.client import BertClient
from multiprocessing import Pool
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim  as optim
import argparse
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from torch.functional import F
import os
import jieba
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output

jieba.load_userdict(os.path.join('data', 'dict.txt.big'))
jieba.add_word('不支持', freq=None, tag=None)

In [ ]:
with open('data/stop_word.txt') as file:
    data = file.read()
stop_words = data.split('\n')

In [ ]:
from sklearn.model_selection import train_test_split
folder = 'data/'
raw_training_data = pd.read_csv(os.path.join(folder,'TD.csv'))
news_urls = pd.read_csv(os.path.join(folder,'NC_1.csv'))
contents = pd.read_json(os.path.join(folder,'url2content.json'), typ=pd.Series)

## sort the contents by index
contents = contents[news_urls.News_URL]
keys, content_list = contents.keys(), contents.values

In [ ]:
import multiprocessing

NumberCPU = multiprocessing.cpu_count()
jieba.initialize()

def jbcut(x):
    if x is not None:
        sen = jieba.lcut(x, cut_all=False)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None
    
pool = multiprocessing.Pool(processes=NumberCPU)
sentenece_arr = pool.map(jbcut,content_list)
pool.close()
pool.join()

In [ ]:
from sklearn.model_selection import train_test_split
batch_size = 100000
folder = 'news_data_1/'
test_query = np.array(pd.read_csv('./data/QS_1.csv').Query)

## okapi model

In [6]:
from gensim.summarization import bm25
bm25Model = bm25.BM25(sentenece_arr)

In [ ]:
train_query = list(set(raw_training_data.Query))

In [ ]:
stands = ['支持', '應該', '反對', '贊成', '不'] 

In [8]:
total_scores = list()
for test_id , text_q in enumerate(test_query):
    text = jieba.lcut(text_q)
    scores = bm25Model.get_scores(text)
    top_query = np.argsort(scores)[::-1][:100]
    
    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))
    
    for query in top_query:
        all_words = [ (sentenece_arr[query][cnt], bm25Model.get_score(sentenece_arr[query], cnt))
                     for cnt, i in enumerate(sentenece_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:20]
        top_words = [x[0] for x in top_words]
        text += top_words
        print(top_words)
    
    scores = bm25Model.get_scores(text)
    total_scores.append(scores)
    
search_result = np.ones((20,300))
for cnt,i in enumerate(total_scores):
    keys = np.argsort(i)[::-1][:300]
    search_result[cnt] += keys
search_result = search_result.astype(np.int)

0 通姦在刑法上應該除罪化 ['通姦', '在', '刑法', '上', '應該', '除罪', '化']
top query num 100
['主播', '並', '國家', '鼓勵', '道德', '「', '提告', '正反', '」', '林奕含', '討論', '專家', '刑事', '反對', '將', '當時', '沒有', '外遇', '\n', '這樣']
['保守', ')', '修法', '認為', '來', '認為', '阿基師', '以還', '說', '國情', '有期徒刑', '團體', '處罰', '\n', '供', '通姦', '通姦', '一樣', '後', '國家']
['還', '男方', '通姦罪', '陳', '處罰', '化法條', '比例', '對象', '\n', '更', '多數', '除罪', '對', '82.2', '於', '調查', '法務部', '必須', '順應', '不必']
['爆發', '除罪', '聽到', '稱職', '扯', '紅燈區', '進去', '引起', '昇', '後', '因為', '委員', '說', '最後', '爆發', '民進黨', '刑法', '昇', '委員會', '檢查']
['感情', '「', '九', '一直', '立委', '家庭', '網友', '條款', '會', '通姦', '來', '規範', '通姦', '國民黨', '中國', '十一年', '通姦罪', '通姦罪', '法律', '並未']
['是否', '常', '介入', '」', '除罪', '讓', '成為', '網友', '\n', '公權力', '〕', '國民黨', '化', '毒舌為', '長期', '吳育', '應', '立委', '牴觸', '卻']
['這項', '過去', '通姦罪', '回應', '為', '社會', '與', '國文', '制度', '性別', '處罰', '法務部', '世界', '共識', '早該', '做出', '刑法', '張筱笛', '國情', '個人']
['「', '〕', '介入', '過', '這個', '行政院', '張俊雄', '轉問', '李', '刑法', '一段話', '不應', '台北', '沒有', '通姦', '

['家庭', '與', '家長', '選上', '候選人', '協會', '沐', '協會', '來', '邀請', '關懷', '家庭', '暴戾之氣', '立委', '工齊', '昨天', '報導', '家庭', '邀請', '台中市']
['還', '包括', '總統府', '年', '任命', '胞妹', '精神疾病', '挑戰', ' ', '代工', '日內瓦', '副部長', '蝦味', '過期', '會', '選舉', '掌', '美設', '穆勒', '人事']
['文貴', '夫妻', '除罪', '比例', '中國', '值得', '與時俱進', '式微', '婚姻制度', '認為', '越南籍', '報導', '宣告', '犯罪', '〔', '為', '認為', '美國', '廢除', '玄奘']
['\n', '通姦', '採訪', '愛情', '歷練', '對方', '三妻四妾', '中', '怎麼', '女性', '\n', '愛情', '頗', '看來', '婚姻', '通姦', '要求', '\n', '一樣', '願意']
['性愛', '諒解', '判刑', '林', '兩人', '400', '記載', '日', '希望', '多次', '法院', '張姓', '個', '〕', '太', '異議', '年', '報導', '對', '\n']
['意義', '人物', '當時', '底層', '性', '私娼', '到場', '性交易', '辯論', '中', '發言人', '行政院', '除罪', '產業', '╱', '越', '化', '推動', '性交易', '交由']
['神經病', '集', '直言', '出軌', '需', '外遇', '判刑', '符合', '〕', '法律', '這麼', '討論', '撤告', '是否', '\n', '出軌', '幾', '表示', '嗎', '必須']
['嚴重性', '〔', '影響', '了解', '後來', '往來', '張女', '認為', '檢方', '西方', '況且', '憤而', '罰金', '〕', '監獄', '罹患', '不大', '年', '小姨子', '後']
['起訴', '送驗', '百分之七十', '例子', '五年', '衛生', '

['工程', '單向', '代管', '強制', '中華民國', '到處', '標誌', '檢討', '車', '道路', '機車', '評估', '內', '公車', '\n', '單向', '先生', '綠燈', '基本', '走']
['想', '校區', '完成', '鄉公所', '也將', '正式', '高雄區', '黃', '練習', '完成', '更能', '考場', '協調', '「', '以往', '不用', '駕訓班', '(', '西嶼', '重型']
['大聲', '肇事', '一地', '「', '我們', '800', '\n', '酒駕', '區', '解體', '\n', '加', '天前', '新北市', '旅車', '過彎', '待轉', '休', '有人', '有點']
['日前', '20', '下午', '林', '歲', '上班', '路口', '當時', '與', '要害', '午休', '對', ')', '今日', '林姓', ' ', '為何', '輛', '返回', '分']
['發生', '是否', '左轉', '格', '全國', '撻伐', '交通', '取消', '陳', '不能', '取締', '\n', '「', '中市', 'Ｔ', '交通安全', '中市', '爭道', '險象環生', '邱素貞']
['不能', '發生', '追撞', '紅線', '區', '憂', '【', '進入', '路口', '反映', '報導', '當做', '最快', '跨越', '與', '左轉', '路口', '路口', '待轉', '機車']
['沒空', '南部', '百八十', '紅燈', '四千多', '違規', '會', '還當', '守法', '熱烈', '飆車族', '這種', '不能', '好像', '\n', '外地', '網友', '很大', '太多', '「']
['」', '字句', '待轉', '好像', '表示', '取締', '依據', '綠燈', '員警', '「', '沒想到', '個', '兩個', '此事', '從中', '指出', '拍照', '時', '左轉', '機車']
['不良', '表示', '議員', '路口', '員鹿路', '員警', '表示', '件', 

['高度', '國內', '交部近', '化解', '產業', '閣揆', '並', '設立', '顯見', '範圍', '朝野', '」', '從', '顧問公司', '博弈', '立法', '不難', '組', '應該', '\n']
['一體', '公投', '僅', '條例', '堅決', '馬祖', '有意', '推動', '為例', '專區', '可說是', '支持', '博弈', '博弈', '排審', '強心針', '管理條例', '爭議', '效果', '卻']
['同意', '意見', '表現', '徵收', '規劃', '研議', '潭', '排除', '花蓮縣', '外部', '發行權', '重大', '記者', '前往', '效應', '博弈', '開放', '太郎', '專用', '本屆']
['議會', '婚姻', '封殺', '有人', '平等', '現在', '如今', '澳洲', '判決', '指出', '特區', '坎培拉', '\n', '可惜', '\n', '無權', '首都', '省', '總理', '聯邦政府']
['附設', '部分', '的確', '未', '籍', '特區', '陳', '國際', '初審', '賭場', '台灣應', '海關', '是否', '態度', '設', '共識', '博弈', '處理', '賭場', '「']
['」', '賭場', '台灣', '貼', '上午', '(', '遞交', '「', '賭場', '指出', '\n', '建設', '園區', '離島', '馬', '來', '場外', '離島', '目前', '國際']
['\n', '前往', '賭場', '生活', '加上', '產業', '建設', '綜合', '禁', '保留', '規劃', '發展', '綁', '觀光', '案', '公共', ' ', '設置', '政府', '馬祖']
['建設', '鬼扯', '評估', '應', 'gunfighter', '如湧', '結合', '警力', '台灣', '馬祖', '專法', '博弈', '\n', '未來', '餐飲', '嗎', '特區', '防止', '業者', '發電']
['文宣', '翁珍', '中立', '時', '說明會', '召集人'

['」', '條之', '交通', '小時', '要求', '12', '書', '小時', '憤怒', '小時', '傾聽', '心聲', '工時', '並', '條款', '年資', '更將', '\n', '當地', '分鐘']
['開票', '協助', '勞動', '預告', '立法院', '盼', '勞動部', '路', '提供', '服員', '\n', '下來', '服', '針對', '過程', '「', '1050623', '罷工', '華航', '會']
['零時', '正常', '出發', '報到', '第一個', '約', '恢復', '回到', '何煖', '力勸', '董事長', '松山', '\n', '華航', '新任', '口號', '何煖', '衝擊', '沒有', '停飛']
['涉犯', '成員', '空服員', '提醒', '一則', '提供', '華航', '已到', '空服員', '交出', '空服員', '資方', '詢問', '前', '航空', '旅客', '獎金', '脅迫', '現場', '收到']
['王', '並', '改善', '糾正', '32', '導致', '合理', '相關', '於', '一致', '並', '\n', '今天', '協調', '嚴重', '各縣', '勞動部', '鑑於', '24', '應']
['投票', '八日', '今晚', '投票', '願意', '降至', '空服員', '於', '華', '還', '報導', '記者會', '記者會', '過', '工廠', '空服員', '出面', '空姐', '喊話', '台北']
['松山機場', '應該', '中', '強度', '1050624', '現身', '表示', '難題', '適合', '休息', '軒', '合適', '何煖', '談', '將和', '軒', '今天下午', '中央社', '軒', '堅持']
['行政院', '全數', '世代', '資產', '短暫', '退休', '儘管', '同時', '休息時間', '地勤', '顯示', '無', '方式', '幾乎', '華', '是否', '日', '新', '輿論', '難得一見']
['秘書長', '更改', '下來', '面對', '影

...

['總統府', '四十三年', '年資', '去年', '」', ')', '哪裡', '十八', '退撫', '傳出', '願意', '退休', '開刀', '「', '十八', '(', '）', '時代', '八', '年資']
['軍公教', '方式', '「', '八成', '％', '召集人', '小組', '有關', '％', '％', '卻', '則是', '設定', '\n', '為', '增加', '立法院', '本俸', '基金', '張哲琛']
['昨在', '兩案', '三', '最晚', '\n', '一是', '兩案', '引起', '％', '調降', '說', '保障', '相同', '歷史', '第一年', '乙案則', '退者', '歸零', '％', '一定']
['\n', '萬多', '年資', '新台幣', '十', '苦哈哈', '趴', '四年', '雙手贊成', '〕', '九', '廿二年', '退', '客社', '農民', '％', '十八', '十八', '月', '錢']
['理所當然', '貪婪', '年資', '退休', '65', '13', '類型', '年資', '這', '工作', '53', '退撫', '直言', '第二類', '們', '父母', '卻', '\n', '非常', '這些']
['歸還', '生活', '即定', '退休', '老年', '基本', '這', '年', '近', '月', '案例', '為', '降到', '退休', '存款', '年', '\n', '(', '本金', '為']
['所得', '轉任', '\n', '人員', '可能', '黨職', '政府', '優存', '領', '蔡', '共識', '退', '對', '部分', '併', '每月', '優存', '得到', '政府', '改革']
['考試院', '\n', '至於', '條款', '退休金', '將從', '\n', '所得', '草案', '細則', '會', '將在', '退休', '18', '調降', '新', '調降', '十五年', '‧', '已退']
['國人', '質疑', '江宜', '了解', '馬', '做出', '利率', '願', '時', '\n

['退', '現在', '慰問金', '嚴重', '多重', '優存', '\n', '政府', '我國', '計算', '退者', '來算', '％', '高達', '單一', '\n', '所得', '第四', '所得', '過度']
['影響', '老', '取消', '退休金', '容易', '所得', '率', ' ', '金化', '退休', '公務員', '八五', '一定', '進步', '拉', '所得', '年金', '差', '勞工', '更']
['」', '讓', '透露', '萬億', '繼續', '第二', '方案', '軍公教', '破壞', '讓', '主導', '\n', '保障', '現階段', '」', '不顧', '最低', '均', '指出', '如今']
['任職', '高層人士', '非', '敷衍', '問題', '各種', '趴', '民進黨', '這種', '三百萬元', '批假', '公職', '這個', '他並', '事務官', '府方', '結束', '減少', '上限', '雖是']
['沒有', '所謂', '保養', '政府', '圖利', '領到', '拿出', '保費', '改善', '七十', '勞保局', '保障', '財政', '\n', '\n', '社會保險', '三柱', '太過分', '年金', '提高']
['必須', '九九', '一萬', '％', '基礎', '公教人員', '億', '政府', '張', '額度', '最多', '對', '津貼', '而後', '並已', '政府', '意見', '公教', '教會', '\n']
['付得', '公務員', '80%', '潛藏', '生活費', '過去', '年金', '相信', '循序漸進', '退', '雖然', '萬元', '打折', '更', '方式', '更是', '同時', '改為', '估單', '達']
['\n', '舊制', '公務人員', '人員', '加台', '目前', '萬名', '七', '公務員', '較', '一年', '九萬', '％', '人員', '調降', '直接', '軍公教', '優存', '乘一', '具有']
['數千', '退撫', '\n', '\n', '增加', 

['犬', '求證', '出動', '物種', '動物', '繁殖', '感染', '說', '書上', '將', '直言', '處', '必須', '直言', '變異', '安樂死', '發病', '米格', '實驗', '狗']
['人員', '實施', '不滿', '狂犬病', '後', '演講時', '進一步', '昨早', '犬', '費昌勇', '承諾', '號召', '昨早', '必要性', '魯犬', '民間', '以外', '民眾', '警方', '一同']
['了解', '台灣', '農委會', '狂犬病', '對', '農委會', '對', '這些', '動物', '做', ')', '病毒', '否則', '特殊', '採用', '病毒', '號', '台灣', '農委會', '陳']
['病毒', '賴比瑞亞', '實驗', '使用', '仍為', '本周', '工商時報', '作出', '使用', '提供', '緊急', '世界衛生組織', '公共衛生', '有效', '賴比瑞亞', '於', '周二', '瑟', '數量', '確認']
['意義', '掛著', '狗', '去函', '人士', '昨', '實驗', '一意孤行', '實驗', '人員', '抗議', '【', '組織', '台灣貓', '昨', '並無', '何足', '十四隻', '想', '昏倒']
['初代', '案', '移植', '生醫', '「', '實驗鼠', '研發', '功用', '成為', '研究', '需要', '國研院', '孕育出', '舉行', '動物', '預計', '兩年', '研究', '\n', '近於零']
['寵物用品', '寵物', '客戶', '企鵝', '穩定', '年', '（', '貿易', '美國', '不紅', '」', '醫院', '供', '商品量', '同業公會', '習慣', '舉例', '品牌', '30', '鄉下']
['不人道', '事項', '做', '台幣', '條之', '多種', '修法', '功能', '產出', '台灣', '物品', '化', '經', '販賣', '尋找', '新', '於', '粧', '管理條例', '相關']
['蘋果', '允許', '動物', '認養', '

['凍漲', '宣布', '做', '油價', '說', '凍漲', '解讀', '元', '「', '」', '下台', '地說', '官員', '考慮', '搶油', '親率', '一點', '持續', '10', '小吃店']
['高達', '虧損', '財務', '當中', '億', '五十', '花費', '為何', '卻', '購買', '改善', '億', '高達', '兩百', '控制', '去年', '說明', '緩漲', '中油', '這也']
['已達', '經濟部長', '無鉛', '將先', '後', '指出', '油價', '下週', '將', '元', '\n', '將', '宣布', '中油', '這', '油價', '漲跌', '75%', '汽油', '30']
['引起', '補貼', '214', '百戶', '耗費', '階層', '來看', '紓', '補貼', '為', '中間', '採行', '那樣', '嗎', '非僅', '\n', '錯', '時', '運輸系統', '導致']
['101', '平均', '石化', '每桶', '負擔', '予以', '配套措施', '油價', '經濟部長', '國際', '柴油', '美元', '昨在', '美元', '挹注', '施顏祥', '設定', '現在', '/', '用油']
['成本', '勒緊褲帶', '虧損', '揭發', '唬爛', '第二', '）', '優先', '開端', '官員', '重整', '騎', '選', '台電', '終端', '台電', '通膨率', '\n', '政府', '比起']
['元', '調漲', '宣布', '天', '\n', '以來', '元', '差價', '98', '差', '一發', '調漲', '「', '大型', '「', '汽油', '但台', '0.9', '柴油', '）']
['則', '鄰國', '後天', '國內', '」', '狂飆', '無鉛', '元', '元', '日', '減', '年', '中油', '\n', '美元', '0.4', '新北市', '中國', '政策', '元']
['現有', '腹案', '目前', '國際', '取消', '模式', '」', '貨物稅', '

['於', '」', '通訊', 'Ｎ', '新政府', '中嘉命', 'Ｃ', '遠傳', '一百七十', '遠傳批', '予以', '三', '購買', '」', '指出', '先前', '啟動', '二', '批評', '二']
['衛生', '廟', '那一刻', '老闆', '不必', '讀冊', '\n', '勇於', 'Ｃ', '」', '夏珍', '什麼', '國共兩黨', '天安門', '客觀事實', '\n', '蔡', '話題', '中時', '地方']
['獲准', '」', '承認', '旺', '中時', '」', '（', '先前', '寬頻', '關係', '要鬥', '因為', '昨日', '聲明', '「', '包括', '我們', '各家', '資產', '誤解']
['太醜', '王小', '提出', '應該', '最後', '\n', '留', '網路上', '集團', '併購案', '(', '故鄉', '」', '進一步', '學者', '」', 'NCC', '掀起', '旺中', '旺中']
['聲援', '傳媒', '調查', '\n', '爭取', '鄭南榕', '副', '學生', '澄清', '情書', '傳出', '反應', '一行', '關注', '敲鼓', '權利', '」', '派', '中國', '「']
['無法挽回', '迄今', '惟', '經營', '從業人員', '文化', '身亡', '卻是', '圈內', '爆張俊彥', '兩大', '飛箭', '有計畫', '壹', '與', '電視', '儘管', '畢露', '商業', '\n']
['\n', '不易', '1.8', '要求', '不要', '壹', '孫', '還', '台灣', '10', '召集人', '蔡衍明', '\n', '案所', '對市', '國龍', '大屠殺', '年初', '台北', '萬人次']
['媒體', '召開', '刊登', '認為', '董事長', '嘉', '結果', '公司', '有線電視', '★', '2011', '網路', '數達', '他們', '消費者', '對', '消費者', '集中', '）', '會']
['邀請', '機制', '媒體', '召開', '旺中', '數

## tfidf model

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(sentenece_arr)
new_corpus = [ dictionary.doc2bow(text) for text in sentenece_arr]

In [ ]:
from gensim.models import TfidfModel
tfidf = TfidfModel(new_corpus)

In [ ]:
doc_dict = {}
for idx, text in enumerate(new_corpus):
    all_words = ([i[0] for i in text])
    
    for word in all_words:
        if word in doc_dict.keys():
            doc_dict[word].append(idx)
        else:
            doc_dict[word]= [idx]

In [ ]:
total_scores = list()

for test_id , text_q in enumerate(test_query):
    text = jieba.lcut(text_q)
    print(test_id ,text_q, text)
    
    text_bow = dictionary.doc2bow(text) 
    keys, values = zip(*tfidf[text_bow])
    
    for cnt, key in enumerate(keys):
        if cnt == 0:
            queried_doc = set(doc_dict[key])
        else:
            queried_doc = set(doc_dict[key]) | queried_doc
    
    cosine_sim_score = list()
    for cnt, doc in enumerate(queried_doc):
        tfidf_score = np.zeros((len(keys)))
        result = sentenece_arr[doc]
        result = dictionary.doc2bow(result)
        result_dict = dict(tfidf[result])
        
        for idx, i in enumerate(keys):
            if i in result_dict.keys():
                tfidf_score[idx] = result_dict[i]
        
        tfidf_score, values  = np.array(tfidf_score).reshape(1,-1), np.array(values).reshape(1,-1)
        score = cosine_similarity(tfidf_score, values)
        cosine_sim_score.append((doc, score[0][0]))
        
    total_scores.append(cosine_sim_score)

In [ ]:
search_result = np.ones((20,300))
for cnt,i in enumerate(total_scores):
    keys , values = zip(*sorted(i, key= lambda x: -x[1])[:300])
    search_result[cnt] += keys
search_result = search_result.astype(np.int)

In [9]:
search_result = search_result.astype(np.int)
df = pd.DataFrame()
df['Query_Index'] = ['q_{:02d}'.format(i+1) for i in range(20)]

for i in range(300):
    df['Rank_{:03d}'.format(i+1)] = search_result[:, i]

for i, row in df.iterrows():
    df.iloc[i, 1:] = df.iloc[i, 1:].apply(lambda x: 'news_{:06d}'.format(x))
    
fname = 'simple13.csv'
df.to_csv('output/' + fname, index=False)

In [ ]:
idx=search_result[1][0]
content_list[idx-1]

## LSTM Model